In [ ]:
pip install torch torchvision torchaudio --upgrade


In [ ]:
import torch
from PIL import Image
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights

ckpt = torch.load("cnn_best.pt", map_location="cpu")
classes = ckpt["classes"]
img_size = ckpt["img_size"]

model = resnet18(weights=None)
in_features = model.fc.in_features
model.fc = torch.nn.Linear(in_features, len(classes))
model.load_state_dict(ckpt["state_dict"])
model.eval()

val_tfms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=ResNet18_Weights.IMAGENET1K_V1.transforms().mean,
                         std=ResNet18_Weights.IMAGENET1K_V1.transforms().std),
])

img = Image.open("some_image.jpg").convert("RGB")
x = val_tfms(img).unsqueeze(0)  # [1,3,H,W]
with torch.no_grad():
    logits = model(x)
    pred = logits.argmax(1).item()

print("Predicted class:", classes[pred])
